<a href="https://colab.research.google.com/github/tylee33/NLP-Lecture/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq
제작: [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) 

1. Chacter level
2. Word level
3. Word levvel + Attention

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)

참고: https://wikidocs.net/24996

In [1]:
!wget https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt

--2021-02-06 03:29:41--  https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: ‘eng-fra.txt’

eng-fra.txt         100%[===================>]   9.10M  44.1MB/s    in 0.2s    

2021-02-06 03:29:41 (44.1 MB/s) - ‘eng-fra.txt’ saved [9541158/9541158]



# 전처리
- special character 제외 
- 영문, 불어 문장 분리

In [2]:
# 다음을 단어로 분리해보자.
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

en_sent = en_sent.replace('-', ' ').replace('?', ' ')
fr_sent = fr_sent.replace('-', ' ').replace('?', ' ')

print(en_sent.lower().split())
print(fr_sent.lower().split())

['have', 'you', 'had', 'dinner']
['avez', 'vous', 'déjà', 'diné']


## 소문자로 변환 후, 특수문자를 찾아서 제거

In [3]:
# 모든 스페셜 캐릭터를 찾아보자.
with open('eng-fra.txt', 'r', encoding='utf-8') as f:
    text = f.read()

spcials = set(text.lower()) - set('abcdefghijklmnopqrstuvwxyzàâçèéêëîïòôöùúûœас\t\n\' ')
print(sorted(spcials))
print(''.join(sorted(spcials)))

['!', '"', '$', '%', '&', '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '~', '\xa0', '«', '\xad', '»', '\u2009', '\u200b', '‐', '–', '‘', '’', '…', '\u202f', '‽', '₂', '€', '\u3000']
!"$%&()+,-./0123456789:;?~ «­» ​‐–‘’… ‽₂€　


In [4]:
print(text[:100])

# 모두 소문자로
text = text.lower()

# 스페셜 캐릭터 제거 
text_alpha = text
for s in spcials:
    text_alpha = text_alpha.replace(s, ' ')

print(text_alpha[:100])

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Sto
go 	va  
run 	cours  
run 	courez  
wow 	ça alors  
fire 	au feu  
help 	à l'aide  
jump 	saute 
sto


## 영어 불어 문장 분리

In [5]:
## 줄로 구분
lines = [line.split('\t') for line in text_alpha.split('\n')]
print(lines[:10])
print(len(lines))

# 50000개 문장만 추출하여 shuffle
import random
lines = lines[:50000]
random.shuffle(lines)

# 영어 문장과 불어 문장을 분리 
eng_sent, fra_sent = zip(*lines)
eng_sent = [s.strip() for s in eng_sent]
fra_sent = [s.strip() for s in fra_sent]
print(eng_sent[:3])
print(fra_sent[:3])

[['go ', 'va  '], ['run ', 'cours  '], ['run ', 'courez  '], ['wow ', 'ça alors  '], ['fire ', 'au feu  '], ['help ', "à l'aide  "], ['jump ', 'saute '], ['stop ', 'ça suffit  '], ['stop ', 'stop  '], ['stop ', 'arrête toi  ']]
135843
['it was a mess', "that's a good guess", 'my house is your house']
["c'était le bordel", 'bien deviné', 'ma demeure est la tienne']


# seq2seq2 I - Charactor Level

1. 데이터 준비
  - tokenizing => 각 문자열에 매핑되는 숫자(인덱스) 구성
  - padding => 길이를 맞춰준다. 
  - indexing => token을 기준으로 실제 문장을 숫자(인덱스)로 변환
  - one hot encoding
1. 모델 준비
  - 모델 생성
  - 모델 학습
1. 모델의 이용
  - 번역


In [6]:
# start 문자열로 탭(\t) 사용
# end 문자열로 줄바꿈(\n) 사용
fra_sent = [f'\t{s}\n'for s in fra_sent]
print(eng_sent[:3])
print(fra_sent[:3])

['it was a mess', "that's a good guess", 'my house is your house']
["\tc'était le bordel\n", '\tbien deviné\n', '\tma demeure est la tienne\n']


## 데이터 준비: tokenizing - 알파벳 하나 하나 단위로 token 삼겠다!

In [7]:
eng_char = sorted(set(''.join(eng_sent)))
fra_char = sorted(set(''.join(fra_sent)))

print(f'영어 - 가장 긴 문장: {max(map(len, eng_sent))}, 사용 알파벳 수: {len(eng_char)}')
print(f'불어 - 가장 긴 문장: {max(map(len, fra_sent))}, 사용 알파벳 수: {len(fra_char)}')

영어 - 가장 긴 문장: 24, 사용 알파벳 수: 29
불어 - 가장 긴 문장: 73, 사용 알파벳 수: 44


In [8]:
eng_token = {c: i for i, c in enumerate(eng_char)}
fra_token = {c: i for i, c in enumerate(fra_char)}

print(eng_token)
print(fra_token)

{' ': 0, "'": 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, 'é': 28}
{'\t': 0, '\n': 1, ' ': 2, "'": 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, 'à': 30, 'â': 31, 'ç': 32, 'è': 33, 'é': 34, 'ê': 35, 'ë': 36, 'î': 37, 'ï': 38, 'ô': 39, 'ù': 40, 'û': 41, 'œ': 42, 'с': 43}


## 데이터 준비: padding => 길이를 맞춰준다.

In [9]:
eng_pad = [list(eng.ljust(24)) for eng in eng_sent]
print(eng_pad[:2])

fra_pad = [list(fra.ljust(73)) for fra in fra_sent]
print(fra_pad[:2])

[['i', 't', ' ', 'w', 'a', 's', ' ', 'a', ' ', 'm', 'e', 's', 's', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], ['t', 'h', 'a', 't', "'", 's', ' ', 'a', ' ', 'g', 'o', 'o', 'd', ' ', 'g', 'u', 'e', 's', 's', ' ', ' ', ' ', ' ', ' ']]
[['\t', 'c', "'", 'é', 't', 'a', 'i', 't', ' ', 'l', 'e', ' ', 'b', 'o', 'r', 'd', 'e', 'l', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], ['\t', 'b', 'i', 'e', 'n', ' ', 'd', 'e', 'v', 'i', 'n', 'é', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


## 데이터 준비: indexing => token을 기준으로 실제 문장을 숫자로 변환

In [10]:
import numpy as np

eng_idx = np.array([list(map(eng_token.get, eng)) for eng in eng_pad])
print(eng_idx[:2])

fra_idx = np.array([list(map(fra_token.get, fra)) for fra in fra_pad])
print(fra_idx[:2])

[[10 21  0 24  2 20  0  2  0 14  6 20 20  0  0  0  0  0  0  0  0  0  0  0]
 [21  9  2 21  1 20  0  2  0  8 16 16  5  0  8 22  6 20 20  0  0  0  0  0]]
[[ 0  6  3 34 23  4 12 23  2 15  8  2  5 18 21  7  8 15  1  2  2  2  2  2
   2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
   2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
   2]
 [ 0  5 12  8 17  2  7  8 25 12 17 34  1  2  2  2  2  2  2  2  2  2  2  2
   2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
   2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
   2]]


## 데이터 준비: one hot encoding

In [11]:
import tensorflow as tf

eng_onehot = tf.keras.utils.to_categorical(eng_idx, num_classes=29)
print(eng_idx.shape, eng_onehot.shape)

fra_onehot = tf.keras.utils.to_categorical(fra_idx, num_classes=44)
print(fra_idx.shape, fra_onehot.shape)

(50000, 24) (50000, 24, 29)
(50000, 73) (50000, 73, 44)


## 모델 준비: 모델 생성


<img src='https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png' width=400>

- (주의) 디코더 쪽의 입력은 fra[:-1]의 형태 출력은 fra[1:]의 형태로 해야함. 
- `dec_X`의 입력 모양이 준비한 `fra_onehot` 실제 모양보다 -1의 된 것에 대한 이해 필요. 
- 학습을 시킬 때에 fra_onehot[:, :-1], fra_onehot[:, 1:] 로 조정하여 입력하게 됨.

In [12]:
import tensorflow as tf

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 영어 - 가장 긴 문장: 24, 사용 알파벳 수: 29, (50000, 24, 29)
# 불어 - 가장 긴 문장: 73, 사용 알파벳 수: 44, (50000, 73, 44)

# Encoder
enc_X = tf.keras.layers.Input(shape=[24, 29])
enc_Y, enc_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(enc_X)

# Decoder
dec_X = tf.keras.layers.Input(shape=[72, 44])
dec_H = tf.keras.layers.GRU(256, return_sequences=True)(dec_X, initial_state=enc_S)
dec_H = tf.keras.layers.Dense(256, activation="swish")(dec_H)
dec_Y = tf.keras.layers.Dense(44, activation="softmax")(dec_H)

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 29)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 72, 44)]     0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 24, 256), (N 220416      input_1[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 72, 256)      231936      input_2[0][0]                    
                                                                 gru[0][1]                    

## 모델 준비: 모델 학습

In [13]:
model.fit([eng_onehot, fra_onehot[:, :-1]], fra_onehot[:, 1:], 
          batch_size=128, epochs=30)

Epoch 1/30
391/391 [==============================] - 14s 15ms/step - loss: 1.0789 - accuracy: 0.7293
Epoch 2/30
391/391 [==============================] - 6s 15ms/step - loss: 0.5867 - accuracy: 0.8210
Epoch 3/30
391/391 [==============================] - 6s 15ms/step - loss: 0.4737 - accuracy: 0.8545
Epoch 4/30
391/391 [==============================] - 6s 15ms/step - loss: 0.4125 - accuracy: 0.8724
Epoch 5/30
391/391 [==============================] - 6s 15ms/step - loss: 0.3744 - accuracy: 0.8841
Epoch 6/30
391/391 [==============================] - 6s 15ms/step - loss: 0.3445 - accuracy: 0.8929
Epoch 7/30
391/391 [==============================] - 6s 15ms/step - loss: 0.3229 - accuracy: 0.8990
Epoch 8/30
391/391 [==============================] - 6s 15ms/step - loss: 0.3075 - accuracy: 0.9038
Epoch 9/30
391/391 [==============================] - 6s 15ms/step - loss: 0.2940 - accuracy: 0.9076
Epoch 10/30
391/391 [==============================] - 6s 15ms/step - loss: 0.2824 - accur

## 모델 이용: 모델을 이용한 번역

In [14]:
import numpy as np

# 영어 - 가장 긴 문장: 24, 사용 알파벳 수: 29
# 불어 - 가장 긴 문장: 73, 사용 알파벳 수: 44

def translate(eng):
    # eng => pad => index => onehot
    eng_sent = list(eng.ljust(24))
    eng_idx = np.array([eng_token.get(c) for c in eng_sent])
    eng_onehot = tf.keras.utils.to_categorical(eng_idx, num_classes=29)

    fra = ''
    for n in range(72):
        # fra => pad => index => onehot
        fra_sent = list(f'\t{fra}'.ljust(72))
        fra_indexed = np.array([fra_token.get(c) for c in fra_sent])
        fra_onehot = tf.keras.utils.to_categorical(fra_indexed, num_classes=44)

        # eng_onehot shape: (24, 29), fra_onehot shape: (72, 44)
        # 3차원 형태로 변형하여 모델에 입력해야 함.
        fra_next = model.predict([eng_onehot.reshape(1, 24, 29), fra_onehot.reshape(1, 72, 44)])

        # onehot -> index -> char
        fra = ''.join([fra_char[i] for i in np.argmax(fra_next[0], axis=1)])
        # 번역된 char 선택
        fra = fra[:n+1]

        if fra[-1] == '\n':
            break

    return fra

In [15]:
import random

# 랜덤 10개
indices = list(range(50000))
random.shuffle(indices)

for n in indices[:10]:
    print(f'영어: {eng_sent[n]}\n불어: {fra_sent[n][1:-1]}')
    print(f'번역: {translate(eng_sent[n])}')

영어: is the bath ready
불어: le bain est il prêt
번역: le chat a t il terminé

영어: tom scrubbed his feet
불어: tom nettoya ses pieds
번역: tom a l'air d'un peu de café

영어: why does he look grumpy
불어: pourquoi a t il l'air grincheux
번역: pourquoi le connaît est il là

영어: i'm not panicking
불어: je ne panique pas
번역: je ne suis pas en train de me faire confiance

영어: stop
불어: arrête toi
번역: arrêtez

영어: stop bothering me
불어: arrête de m'ennuyer
번역: arrêtez de m'en alles

영어: i'm sick of lying
불어: j'en ai marre de mentir
번역: j'ai des problèmes

영어: he cannot be saved
불어: il ne peut être sauvé
번역: il ne peut pas le faire ici

영어: he lifted her in the air
불어: il la souleva en l'air
번역: il est allé à la porte ouverte

영어: what're you good at
불어: à quoi êtes vous bon
번역: que cherchez vous



# Seq2Seq II - Word Level (without attention)
1. 데이터 준비
  - 문장을 단어로 분리 
  - tokeninzing - 각 단어에 매핑되는 숫자(인덱스) 구성
  - indexing
  - padding
1. 모델 준비 
  - 모델 생성
  - 모델 학습
1. 모델의 이용
  - 번역

## 데이터 준비: 문장을 단어를 기준으로 분리

In [16]:
eng_words = [eng.split() for eng in eng_sent]
# start 단어로 '<sos>' 사용, end 단어로 '<eos>' 사용
fra_words = [f'<sos> {fra} <eos>'.split() for fra in fra_sent]

print(eng_words[:3])
print(fra_words[:3])

[['it', 'was', 'a', 'mess'], ["that's", 'a', 'good', 'guess'], ['my', 'house', 'is', 'your', 'house']]
[['<sos>', "c'était", 'le', 'bordel', '<eos>'], ['<sos>', 'bien', 'deviné', '<eos>'], ['<sos>', 'ma', 'demeure', 'est', 'la', 'tienne', '<eos>']]


## 데이터 준비: tokenizing & indexing & padding

In [17]:
import tensorflow as tf

# tokenizing
tokenizer_en = tf.keras.preprocessing.text.Tokenizer()
tokenizer_en.fit_on_texts(eng_words)

tokenizer_fr = tf.keras.preprocessing.text.Tokenizer()
tokenizer_fr.fit_on_texts(fra_words)

# indexing
eng_idx = tokenizer_en.texts_to_sequences(eng_words)
fra_idx = tokenizer_fr.texts_to_sequences(fra_words)

print(eng_idx[:3])
print(fra_idx[:3])

# padding
eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_idx, padding="post")
fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_idx, padding="post")

print(eng_pad.shape)
print(fra_pad.shape)

[[7, 23, 3, 836], [50, 3, 65, 390], [21, 171, 4, 22, 171]]
[[1, 123, 11, 1889, 2], [1, 54, 1740, 2], [1, 61, 1741, 10, 15, 1423, 2]]
(50000, 8)
(50000, 16)


In [18]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

enc_vocab_size = len(tokenizer_en.word_index) + 1
fra_vocab_size = len(tokenizer_fr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n프랑스어 단어 집합의 크기: {:d}".format(enc_vocab_size, fra_vocab_size))

영어 단어 집합의 크기: 6090
프랑스어 단어 집합의 크기: 11804


## 데이터 준비: 원핫인코딩
- **메모리가 터짐**
- 다른 대안은 없을까? => Embedding
- https://wikidocs.net/33520
- https://www.tensorflow.org/tutorials/text/word_embeddings#using_the_embedding_layer

In [19]:
# import tensorflow as tf

# eng_onehot = tf.keras.utils.to_categorical(eng_pad, num_classes=6090)
# print(eng_pad.shape, eng_onehot.shape)

# fra_onehot = tf.keras.utils.to_categorical(fra_pad, num_classes=11804)
# print(fra_pad.shape, fra_onehot.shape)

## 모델 준비: 모델 생성 - 기본 구조 (without attention)

<img src='https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png' width=400>

- (주의) 디코더 쪽의 입력은 fra[:-1]의 형태 출력은 fra[1:]의 형태로 해야함. 
- `dec_X`의 입력 모양이 준비한 `fra_pad` 실제 모양보다 -1의 된 것에 대한 이해 필요. 
- 학습을 시킬 때에 fra_pad[:, :-1], fra_pad[:, 1:] 로 조정하여 입력하게 됨.
- label이 onehot encoding이 되어있지 않은 경우에, loss를 `sparse_categorical_crossentropy` 이용

In [20]:
import tensorflow as tf

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 영어 단어 집합의 크기: 6090, (50000, 8)
# 프랑스어 단어 집합의 크기: 11804, (50000, 16)

# Encoder
enc_X = tf.keras.layers.Input(shape=[8])
enc_E = tf.keras.layers.Embedding(6090, 50)(enc_X) # 토큰수, 차원수
enc_Y, enc_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(enc_E)

# Decoder
dec_X = tf.keras.layers.Input(shape=[15])
dec_E = tf.keras.layers.Embedding(11804, 50)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.GRU(256, return_sequences=True)(dec_E, initial_state=enc_S)
dec_H = tf.keras.layers.Dense(256, activation="swish")(dec_H)
dec_Y = tf.keras.layers.Dense(11804, activation="softmax")(dec_H)

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 8, 50)        304500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 50)       590200      input_2[0][0]                    
______________________________________________________________________________________________

## 모델 준비: 모델 학습

In [21]:
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], 
          batch_size=128, epochs=5)

Epoch 1/5
391/391 [==============================] - 17s 38ms/step - loss: 2.7050 - accuracy: 0.6569
Epoch 2/5
391/391 [==============================] - 15s 37ms/step - loss: 1.6590 - accuracy: 0.7406
Epoch 3/5
391/391 [==============================] - 15s 37ms/step - loss: 1.4729 - accuracy: 0.7627
Epoch 4/5
391/391 [==============================] - 15s 38ms/step - loss: 1.3539 - accuracy: 0.7774
Epoch 5/5
391/391 [==============================] - 15s 38ms/step - loss: 1.2641 - accuracy: 0.7882


## 모델 이용: 모델을 이용한 번역

In [22]:
import numpy as np

# 영어 단어 집합의 크기: 6090, (50000, 8)
# 프랑스어 단어 집합의 크기: 11804, (50000, 16)

def translate(eng):
    # eng => index => pad
    eng_idx = tokenizer_en.texts_to_sequences([eng])
    eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_idx, maxlen=8, padding="post")

    fra = []
    for n in range(15):
        # fra => index => pad
        fra_index = tokenizer_fr.texts_to_sequences([['<sos>'] + fra])
        fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_index, maxlen=15, padding="post")

        fra_next = model.predict([eng_pad, fra_pad])

        # onehot -> index -> word
        fra = [tokenizer_fr.index_word[i] for i in np.argmax(fra_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        fra = fra[:n+1]
        
        if fra[-1] == '<eos>':
            break

    return fra

In [23]:
import random

# 랜덤 10개
indices = list(range(50000))
random.shuffle(indices)

for n in indices[:10]:
    print(f"영어: {' '.join(eng_words[n])}\n불어: {' '.join(fra_words[n][1:-1])}")
    print(f"번역: {' '.join(translate(eng_words[n])[:-1])}")
    print()


영어: i'll be glad to
불어: oui avec joie
번역: je suis en train de vous

영어: tom wanted to help mary
불어: tom voulait aider mary
번역: tom a fait un peu

영어: they're not following
불어: elles ne sont pas en train de suivre
번역: elles ne sont pas si

영어: i owe him my life
불어: je lui dois la vie
번역: je vous ai un peu de la maison

영어: you're still green
불어: tu es encore un bleu
번역: vous êtes très très heureux

영어: the car bumped the tree
불어: la voiture a heurté l'arbre
번역: le chien est en train de la maison

영어: i slept very well
불어: je dormis fort bien
번역: je me suis senti très heureux

영어: i closed my umbrella
불어: je fermai mon parapluie
번역: je me suis senti très heureux

영어: maybe i'll call you
불어: peut être t'appellerai je
번역: ce n'est pas si

영어: can i sit next to you
불어: est ce que je peux m'asseoir à côté de toi
번역: puis je vous voir



# Seq2Seq II - Word Level (with attention)
1. 모델 준비 (위 모델을 이용한 개선)
  - 모델 생성
  - 모델 학습


## 모델 준비: 모델 생성 - Attention 구조 (dot prodoct ver)
- 커멘트의 설명은 **딥 러닝을 이용한 자연어 처리 입문** > **어텐션 메커니즘 (Attention Mechanism)** 문서를 참고했습니다. 
  1. 어텐션 스코어(Attention Score)를 구한다.
  1. 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.
  1. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.
  1. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)
  1. 출력층 연산의 입력이 되는 dec_H를 계산합니다.
  - https://wikidocs.net/22893

In [24]:
import tensorflow as tf

# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 영어 단어 집합의 크기: 6090, (50000, 8)
# 프랑스어 단어 집합의 크기: 11804, (50000, 16)

# Encoder
enc_X = tf.keras.layers.Input(shape=[8])
enc_E = tf.keras.layers.Embedding(6090, 50)(enc_X) # 토큰수, 차원수
enc_Y, enc_S = tf.keras.layers.SimpleRNN(256, return_sequences=True, return_state=True)(enc_E)

# Decoder
dec_X = tf.keras.layers.Input(shape=[15])
dec_E = tf.keras.layers.Embedding(11804, 50)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.SimpleRNN(256, return_sequences=True)(dec_E, initial_state=enc_S)

################################
## attention layer
################################
# key, value, query
key = enc_Y  # 인코더의 히든스테이트를 key로 활용한다. (8, 256)
value = enc_Y  # 인코더의 히든스테이트를 value로 활용한다. (8, 256)
query = dec_H  # 디코더의 히든스테이트를 query로 활용한다. (15, 256)

# 1. 어텐션 스코어(Attention Score)를 구한다.
score = tf.matmul(query, key, transpose_b=True)
# 연산 결과: (15, 256) * (256, 8) => (15, 8)

# 2. 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.
att_dist = tf.nn.softmax(score, axis=-1) 

# 3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.
att_value = tf.matmul(att_dist, value)
# 연산 결과: (15, 8) * (8, 256) => (15, 256)

# 4. 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결한다.(Concatenate)
dec_H = tf.keras.layers.Concatenate()([att_value, dec_H])

# 5. 출력층 연산의 입력이 되는 dec_H를 계산합니다.
dec_H = tf.keras.layers.Dense(256, activation='tanh')(dec_H)
################################

dec_Y = tf.keras.layers.Dense(11804, activation="softmax")(dec_H)

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 8, 50)        304500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 50)       590200      input_2[0][0]                    
______________________________________________________________________________________________

## 모델 준비: 모델 학습

In [25]:
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], 
          batch_size=128, epochs=30)

Epoch 1/30
391/391 [==============================] - 18s 43ms/step - loss: 2.5827 - accuracy: 0.6658
Epoch 2/30
391/391 [==============================] - 17s 43ms/step - loss: 1.4841 - accuracy: 0.7716
Epoch 3/30
391/391 [==============================] - 17s 43ms/step - loss: 1.3049 - accuracy: 0.7925
Epoch 4/30
391/391 [==============================] - 17s 43ms/step - loss: 1.1940 - accuracy: 0.8068
Epoch 5/30
391/391 [==============================] - 17s 43ms/step - loss: 1.1040 - accuracy: 0.8181
Epoch 6/30
391/391 [==============================] - 17s 43ms/step - loss: 1.0381 - accuracy: 0.8265
Epoch 7/30
391/391 [==============================] - 17s 43ms/step - loss: 0.9774 - accuracy: 0.8348
Epoch 8/30
391/391 [==============================] - 17s 43ms/step - loss: 0.9274 - accuracy: 0.8418
Epoch 9/30
391/391 [==============================] - 17s 43ms/step - loss: 0.8858 - accuracy: 0.8484
Epoch 10/30
391/391 [==============================] - 17s 43ms/step - loss: 0.849

## 모델 이용: 모델을 이용한 번역

In [26]:
import numpy as np

# 영어 단어 집합의 크기: 6090, (50000, 8)
# 프랑스어 단어 집합의 크기: 11804, (50000, 16)

def translate(eng):
    # eng => index => pad
    eng_idx = tokenizer_en.texts_to_sequences([eng])
    eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_idx, maxlen=8, padding="post")

    fra = []
    for n in range(15):
        # fra => index => pad
        fra_index = tokenizer_fr.texts_to_sequences([['<sos>'] + fra])
        fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_index, maxlen=15, padding="post")

        fra_next = model.predict([eng_pad, fra_pad])

        # onehot -> index -> word
        fra = [tokenizer_fr.index_word[i] for i in np.argmax(fra_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        fra = fra[:n+1]

        if fra[-1] == '<eos>':
            break

    return fra

In [27]:
import random

# 랜덤 10개
indices = list(range(50000))
random.shuffle(indices)

for n in indices[:10]:
    print(f"영어: {' '.join(eng_words[n])}\n불어: {' '.join(fra_words[n][1:-1])}")
    print(f"번역: {' '.join(translate(eng_words[n])[:-1])}")
    print()


영어: it feels like a dream
불어: on se sent comme dans un rêve
번역: on se sent une très pris

영어: treat her well
불어: traite la bien
번역: cela se passe

영어: tom is also an artist
불어: tom est aussi artiste
번역: tom est aussi jeune

영어: it was almost funny
불어: c'était presque amusant
번역: c'était presque amusant

영어: i'll go look
불어: j'irai voir
번역: j'irai en un

영어: you leave me no choice
불어: vous ne me laissez aucun choix
번역: tu ne me le aucun choix

영어: we all deserve to go
불어: nous méritons tous d'y aller
번역: nous voulons tous partir

영어: everyone looks worried
불어: tout le monde a l'air soucieux
번역: tout le monde a l'air malade

영어: is that a promise
불어: s'agit il d'une promesse
번역: s'agit il d'une quelque chose

영어: my life is in your hands
불어: ma vie repose entre vos mains
번역: ma vie est sur lui peur

